# Practical example. Audiobooks

## Create the machine learning algorithm, Accuracy distribution

### Import the relevant libraries

In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

### Data

In [2]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [3]:

max_epochs = 20
batch_size = 64

input_size = 10
output_size = 2

In [4]:

HP_LAMBDA_REG = hp.HParam('lambda', hp.Discrete([5e-5, 1e-4]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.0, 0.1, 0.2]))
HP_RUN_NUM = hp.HParam('run number', hp.Discrete([i + 1 for i in range(10)]))

with tf.summary.create_file_writer(r'Logs/Model 3/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_LAMBDA_REG, HP_DROPOUT, HP_RUN_NUM],
        metrics = [hp.Metric('accuracy', display_name = 'Accuracy')]
    )


In [5]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, 'relu',
                              kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_LAMBDA_REG])),
        
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        
        tf.keras.layers.Dense(128, 'relu',
                              kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_LAMBDA_REG])),
        
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        
        tf.keras.layers.Dense(output_size, activation='softmax')
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(name='sparse_categorical_crossentropy')
    
    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['sparse_categorical_crossentropy', 'accuracy'])
    
    # Defining the logging directory
    log_dir = r'Logs/Model 3/fit/' + 'run-{}'.format(session_num)

    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')     
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(validation_inputs)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(validation_targets, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["won't buy", "will buy"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_sparse_categorical_crossentropy', # We should supply the unregularized the loss
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        train_inputs,
        train_targets,
        epochs = max_epochs,
        batch_size = batch_size,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (validation_inputs, validation_targets),
        verbose = 2
    )
    
    
    # Evaluating the model's performance on the validation set
    # Note that we have three metrics 'see sparse_categorical_crossentropy'
    _, __, accuracy = model.evaluate(validation_inputs, validation_targets)
    
    # Saving the current model for future reference
    model.save(r'saved_models\Model 3\Run-{}'.format(session_num))
    
    return accuracy


In [6]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar('accuracy', accuracy, step=1)


In [7]:
session_num = 1

for lambda_reg in HP_LAMBDA_REG.domain.values:
    for dropout in HP_DROPOUT.domain.values:
        for run_number in HP_RUN_NUM.domain.values:
        
            hparams = {
                HP_LAMBDA_REG: lambda_reg,
                HP_DROPOUT: dropout,
                HP_RUN_NUM: run_number
            }

            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('Logs/Model 3/hparam_tuning/' + run_name, hparams, session_num)

            session_num += 1


--- Starting trial: run-1
{'lambda': 5e-05, 'dropout': 0.0, 'run number': 1}
Epoch 1/20
56/56 - 2s - loss: 0.4148 - sparse_categorical_crossentropy: 0.4075 - accuracy: 0.8483 - val_loss: 0.3017 - val_sparse_categorical_crossentropy: 0.2944 - val_accuracy: 0.8904
Epoch 2/20
56/56 - 1s - loss: 0.3018 - sparse_categorical_crossentropy: 0.2945 - accuracy: 0.8896 - val_loss: 0.2696 - val_sparse_categorical_crossentropy: 0.2622 - val_accuracy: 0.9038
Epoch 3/20
56/56 - 1s - loss: 0.2831 - sparse_categorical_crossentropy: 0.2757 - accuracy: 0.9008 - val_loss: 0.2686 - val_sparse_categorical_crossentropy: 0.2612 - val_accuracy: 0.9016
Epoch 4/20
56/56 - 1s - loss: 0.2714 - sparse_categorical_crossentropy: 0.2641 - accuracy: 0.8980 - val_loss: 0.2542 - val_sparse_categorical_crossentropy: 0.2468 - val_accuracy: 0.9083
Epoch 5/20
56/56 - 1s - loss: 0.2615 - sparse_categorical_crossentropy: 0.2541 - accuracy: 0.9022 - val_loss: 0.2626 - val_sparse_categorical_crossentropy: 0.2552 - val_accuracy: 

INFO:tensorflow:Assets written to: saved_models\Model 3\Run-4\assets
--- Starting trial: run-5
{'lambda': 5e-05, 'dropout': 0.0, 'run number': 5}
Epoch 1/20
56/56 - 2s - loss: 0.4112 - sparse_categorical_crossentropy: 0.4038 - accuracy: 0.8569 - val_loss: 0.2971 - val_sparse_categorical_crossentropy: 0.2897 - val_accuracy: 0.8859
Epoch 2/20
56/56 - 1s - loss: 0.3003 - sparse_categorical_crossentropy: 0.2928 - accuracy: 0.8913 - val_loss: 0.2719 - val_sparse_categorical_crossentropy: 0.2644 - val_accuracy: 0.9038
Epoch 3/20
56/56 - 1s - loss: 0.2791 - sparse_categorical_crossentropy: 0.2716 - accuracy: 0.8966 - val_loss: 0.2682 - val_sparse_categorical_crossentropy: 0.2607 - val_accuracy: 0.9016
Epoch 4/20
56/56 - 1s - loss: 0.2735 - sparse_categorical_crossentropy: 0.2660 - accuracy: 0.9000 - val_loss: 0.2566 - val_sparse_categorical_crossentropy: 0.2491 - val_accuracy: 0.9128
Epoch 5/20
56/56 - 1s - loss: 0.2696 - sparse_categorical_crossentropy: 0.2621 - accuracy: 0.9011 - val_loss: 

14/14 [==============================] - 0s 2ms/step - loss: 0.2618 - sparse_categorical_crossentropy: 0.2543 - accuracy: 0.9038
INFO:tensorflow:Assets written to: saved_models\Model 3\Run-9\assets
--- Starting trial: run-10
{'lambda': 5e-05, 'dropout': 0.0, 'run number': 10}
Epoch 1/20
56/56 - 2s - loss: 0.4067 - sparse_categorical_crossentropy: 0.3992 - accuracy: 0.8558 - val_loss: 0.2975 - val_sparse_categorical_crossentropy: 0.2900 - val_accuracy: 0.8837
Epoch 2/20
56/56 - 1s - loss: 0.3061 - sparse_categorical_crossentropy: 0.2986 - accuracy: 0.8888 - val_loss: 0.2702 - val_sparse_categorical_crossentropy: 0.2627 - val_accuracy: 0.9038
Epoch 3/20
56/56 - 1s - loss: 0.2840 - sparse_categorical_crossentropy: 0.2765 - accuracy: 0.8977 - val_loss: 0.2661 - val_sparse_categorical_crossentropy: 0.2586 - val_accuracy: 0.9016
Epoch 4/20
56/56 - 1s - loss: 0.2755 - sparse_categorical_crossentropy: 0.2680 - accuracy: 0.9008 - val_loss: 0.2671 - val_sparse_categorical_crossentropy: 0.2596 - 

Epoch 8/20
56/56 - 1s - loss: 0.2638 - sparse_categorical_crossentropy: 0.2563 - accuracy: 0.9039 - val_loss: 0.2441 - val_sparse_categorical_crossentropy: 0.2367 - val_accuracy: 0.9150
Epoch 9/20
56/56 - 1s - loss: 0.2601 - sparse_categorical_crossentropy: 0.2527 - accuracy: 0.9047 - val_loss: 0.2440 - val_sparse_categorical_crossentropy: 0.2366 - val_accuracy: 0.9083
Epoch 10/20
56/56 - 1s - loss: 0.2548 - sparse_categorical_crossentropy: 0.2474 - accuracy: 0.9039 - val_loss: 0.2416 - val_sparse_categorical_crossentropy: 0.2342 - val_accuracy: 0.9128
Epoch 11/20
56/56 - 1s - loss: 0.2590 - sparse_categorical_crossentropy: 0.2516 - accuracy: 0.9044 - val_loss: 0.2451 - val_sparse_categorical_crossentropy: 0.2377 - val_accuracy: 0.9128
Epoch 12/20
56/56 - 1s - loss: 0.2545 - sparse_categorical_crossentropy: 0.2471 - accuracy: 0.9053 - val_loss: 0.2418 - val_sparse_categorical_crossentropy: 0.2344 - val_accuracy: 0.9150
14/14 [==============================] - 0s 2ms/step - loss: 0.2416

56/56 - 1s - loss: 0.3110 - sparse_categorical_crossentropy: 0.3034 - accuracy: 0.8880 - val_loss: 0.2780 - val_sparse_categorical_crossentropy: 0.2704 - val_accuracy: 0.8971
Epoch 4/20
56/56 - 1s - loss: 0.2934 - sparse_categorical_crossentropy: 0.2857 - accuracy: 0.8916 - val_loss: 0.2598 - val_sparse_categorical_crossentropy: 0.2521 - val_accuracy: 0.9038
Epoch 5/20
56/56 - 1s - loss: 0.2847 - sparse_categorical_crossentropy: 0.2771 - accuracy: 0.8977 - val_loss: 0.2568 - val_sparse_categorical_crossentropy: 0.2492 - val_accuracy: 0.9038
Epoch 6/20
56/56 - 1s - loss: 0.2823 - sparse_categorical_crossentropy: 0.2747 - accuracy: 0.8969 - val_loss: 0.2592 - val_sparse_categorical_crossentropy: 0.2516 - val_accuracy: 0.9060
Epoch 7/20
56/56 - 1s - loss: 0.2802 - sparse_categorical_crossentropy: 0.2725 - accuracy: 0.8991 - val_loss: 0.2506 - val_sparse_categorical_crossentropy: 0.2430 - val_accuracy: 0.9060
Epoch 8/20
56/56 - 1s - loss: 0.2770 - sparse_categorical_crossentropy: 0.2694 - 

...

Epoch 3/20
56/56 - 1s - loss: 0.2884 - sparse_categorical_crossentropy: 0.2743 - accuracy: 0.8975 - val_loss: 0.2639 - val_sparse_categorical_crossentropy: 0.2499 - val_accuracy: 0.9060
Epoch 4/20
56/56 - 1s - loss: 0.2767 - sparse_categorical_crossentropy: 0.2628 - accuracy: 0.9028 - val_loss: 0.2557 - val_sparse_categorical_crossentropy: 0.2418 - val_accuracy: 0.9060
Epoch 5/20
56/56 - 1s - loss: 0.2707 - sparse_categorical_crossentropy: 0.2569 - accuracy: 0.9016 - val_loss: 0.2541 - val_sparse_categorical_crossentropy: 0.2403 - val_accuracy: 0.9128
Epoch 6/20
56/56 - 1s - loss: 0.2657 - sparse_categorical_crossentropy: 0.2520 - accuracy: 0.9047 - val_loss: 0.2545 - val_sparse_categorical_crossentropy: 0.2408 - val_accuracy: 0.9083
Epoch 7/20
56/56 - 1s - loss: 0.2631 - sparse_categorical_crossentropy: 0.2494 - accuracy: 0.9044 - val_loss: 0.2545 - val_sparse_categorical_crossentropy: 0.2409 - val_accuracy: 0.9060
14/14 [==============================] - 0s 1ms/step - loss: 0.2541 - 

Epoch 9/20
56/56 - 1s - loss: 0.2726 - sparse_categorical_crossentropy: 0.2586 - accuracy: 0.9039 - val_loss: 0.2581 - val_sparse_categorical_crossentropy: 0.2441 - val_accuracy: 0.9060
14/14 [==============================] - 0s 3ms/step - loss: 0.2540 - sparse_categorical_crossentropy: 0.2398 - accuracy: 0.9083
INFO:tensorflow:Assets written to: saved_models\Model 3\Run-42\assets
--- Starting trial: run-43
{'lambda': 0.0001, 'dropout': 0.1, 'run number': 3}
Epoch 1/20
56/56 - 2s - loss: 0.4274 - sparse_categorical_crossentropy: 0.4129 - accuracy: 0.8293 - val_loss: 0.3108 - val_sparse_categorical_crossentropy: 0.2963 - val_accuracy: 0.8881
Epoch 2/20
56/56 - 1s - loss: 0.3206 - sparse_categorical_crossentropy: 0.3062 - accuracy: 0.8868 - val_loss: 0.2785 - val_sparse_categorical_crossentropy: 0.2644 - val_accuracy: 0.8993
Epoch 3/20
56/56 - 1s - loss: 0.3001 - sparse_categorical_crossentropy: 0.2861 - accuracy: 0.8902 - val_loss: 0.2712 - val_sparse_categorical_crossentropy: 0.2573 -

Epoch 5/20
56/56 - 1s - loss: 0.2969 - sparse_categorical_crossentropy: 0.2825 - accuracy: 0.8947 - val_loss: 0.2624 - val_sparse_categorical_crossentropy: 0.2480 - val_accuracy: 0.9038
Epoch 6/20
56/56 - 1s - loss: 0.2935 - sparse_categorical_crossentropy: 0.2792 - accuracy: 0.8991 - val_loss: 0.2627 - val_sparse_categorical_crossentropy: 0.2484 - val_accuracy: 0.9038
Epoch 7/20
56/56 - 1s - loss: 0.2853 - sparse_categorical_crossentropy: 0.2712 - accuracy: 0.8963 - val_loss: 0.2524 - val_sparse_categorical_crossentropy: 0.2383 - val_accuracy: 0.9150
Epoch 8/20
56/56 - 1s - loss: 0.2846 - sparse_categorical_crossentropy: 0.2706 - accuracy: 0.8989 - val_loss: 0.2532 - val_sparse_categorical_crossentropy: 0.2393 - val_accuracy: 0.9128
Epoch 9/20
56/56 - 1s - loss: 0.2781 - sparse_categorical_crossentropy: 0.2643 - accuracy: 0.9033 - val_loss: 0.2546 - val_sparse_categorical_crossentropy: 0.2408 - val_accuracy: 0.9083
14/14 [==============================] - ETA: 0s - loss: 0.3171 - spar

Epoch 7/20
56/56 - 1s - loss: 0.2847 - sparse_categorical_crossentropy: 0.2706 - accuracy: 0.8975 - val_loss: 0.2628 - val_sparse_categorical_crossentropy: 0.2488 - val_accuracy: 0.9060
Epoch 8/20
56/56 - 1s - loss: 0.2827 - sparse_categorical_crossentropy: 0.2687 - accuracy: 0.9022 - val_loss: 0.2528 - val_sparse_categorical_crossentropy: 0.2388 - val_accuracy: 0.9105
Epoch 9/20
56/56 - 1s - loss: 0.2745 - sparse_categorical_crossentropy: 0.2606 - accuracy: 0.9033 - val_loss: 0.2531 - val_sparse_categorical_crossentropy: 0.2392 - val_accuracy: 0.9083
Epoch 10/20
56/56 - 1s - loss: 0.2760 - sparse_categorical_crossentropy: 0.2622 - accuracy: 0.9014 - val_loss: 0.2588 - val_sparse_categorical_crossentropy: 0.2450 - val_accuracy: 0.9060
14/14 [==============================] - 0s 2ms/step - loss: 0.2528 - sparse_categorical_crossentropy: 0.2388 - accuracy: 0.9105
INFO:tensorflow:Assets written to: saved_models\Model 3\Run-55\assets
--- Starting trial: run-56
{'lambda': 0.0001, 'dropout':

Epoch 14/20
56/56 - 1s - loss: 0.2691 - sparse_categorical_crossentropy: 0.2563 - accuracy: 0.9030 - val_loss: 0.2501 - val_sparse_categorical_crossentropy: 0.2372 - val_accuracy: 0.9105
Epoch 15/20
56/56 - 1s - loss: 0.2658 - sparse_categorical_crossentropy: 0.2530 - accuracy: 0.9075 - val_loss: 0.2458 - val_sparse_categorical_crossentropy: 0.2330 - val_accuracy: 0.9128
Epoch 16/20
56/56 - 1s - loss: 0.2575 - sparse_categorical_crossentropy: 0.2448 - accuracy: 0.9092 - val_loss: 0.2509 - val_sparse_categorical_crossentropy: 0.2382 - val_accuracy: 0.9150
Epoch 17/20
56/56 - 1s - loss: 0.2677 - sparse_categorical_crossentropy: 0.2551 - accuracy: 0.9056 - val_loss: 0.2410 - val_sparse_categorical_crossentropy: 0.2285 - val_accuracy: 0.9172
Epoch 18/20
56/56 - 1s - loss: 0.2584 - sparse_categorical_crossentropy: 0.2459 - accuracy: 0.9064 - val_loss: 0.2559 - val_sparse_categorical_crossentropy: 0.2434 - val_accuracy: 0.9128
Epoch 19/20
56/56 - 1s - loss: 0.2618 - sparse_categorical_crosse

In [4]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 3/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4956), started 0:00:57 ago. (Use '!kill 4956' to kill it.)

In [4]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 3/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9020), started 0:01:02 ago. (Use '!kill 9020' to kill it.)

## Test the model

In [5]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models\Model 3\Run-33")

In [8]:
test_pred = np.argmax(model.predict(test_inputs), axis=1)
test_accuracy = (test_targets == test_pred).sum() / test_targets.shape[0]

In [9]:
print('\nTest accuracy: {0:.2f}%'.format(test_accuracy * 100.))


Test accuracy: 91.52%
